In [48]:
import time
import json
from pathlib import Path
import pandas as pd
from pandas.io.formats.style import Styler
from collections.abc import Generator, Callable
import typing
from typing import Any, TypeAlias
import numpy as np
from contextlib import contextmanager
from functools import partial, reduce
import re
import datetime as dt
from tqdm import tqdm
import pickle
from IPython.display import (
    display, # type: ignore[reportUnknownVariableType]
    Markdown,
)
import importlib
import spacy

from config.fastf1 import fastf1
import fastf1.events as fastf1_events
from config import config
importlib.reload(config);
from src.data.loader import stream_ndjson, load_submissions_df, load_comments_df
import src.data.preprocessing as preprocessing
importlib.reload(preprocessing);
import src.data.constants as dataset_constants
import src.utils
importlib.reload(src.utils);
from src.utils import (
    temporary_pandas_options,
    display_full_dataframe,
    hide_index,
    compose,
)
from src import utils
utils.set_random_seeds()

import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)

DEVICE = utils.get_device()

PyTorch version: 2.5.1+cu124
CUDA available: True
CUDA version: 12.4
Selected GPU: NVIDIA GeForce GTX 1080 Ti (device_id=0)


In [49]:
def load_f1_df(limit: int | None = None, in_place: bool = True) -> pd.DataFrame:
    ndjson_streamer = partial(stream_ndjson, limit=limit)

    return preprocessing.concatenate_submissions_and_comments(
        submissions_df=load_submissions_df(dataset_constants.RawFile.FORMULA1_SUBMISSIONS, ndjson_streamer),
        comments_df=load_comments_df(dataset_constants.RawFile.FORMULA1_COMMENTS, ndjson_streamer),
        in_place=in_place,
    )

def load_f15_df(limit: int | None = None, in_place: bool = True) -> pd.DataFrame:
    ndjson_streamer = partial(stream_ndjson, limit=limit)

    return preprocessing.concatenate_submissions_and_comments(
        submissions_df=load_submissions_df(dataset_constants.RawFile.FORMULA1POINT5_SUBMISSIONS, ndjson_streamer),
        comments_df=load_comments_df(dataset_constants.RawFile.FORMULA1POINT5_COMMENTS, ndjson_streamer),
        in_place=in_place,
    )

In [50]:
f1_ndjson_streamer = partial(stream_ndjson, limit=100)
f15_ndjson_streamer = partial(stream_ndjson, limit=100)

f1_submissions_df = load_submissions_df(dataset_constants.RawFile.FORMULA1_SUBMISSIONS, f1_ndjson_streamer)
f1_comments_df = load_comments_df(dataset_constants.RawFile.FORMULA1_COMMENTS, f1_ndjson_streamer)

f15_submissions_df = load_submissions_df(dataset_constants.RawFile.FORMULA1POINT5_SUBMISSIONS, f15_ndjson_streamer)
f15_comments_df = load_comments_df(dataset_constants.RawFile.FORMULA1POINT5_COMMENTS, f15_ndjson_streamer)

f1_df = preprocessing.concatenate_submissions_and_comments(f1_submissions_df, f1_comments_df)
f15_df = preprocessing.concatenate_submissions_and_comments(f15_submissions_df, f15_comments_df)

In [51]:
n = 4

with display_full_dataframe():
    display(Markdown('### r/formula1 submissions:'), f1_submissions_df.head(n))
    display(Markdown('### r/formula1 comments:'), f1_comments_df.head(n))
    display(Markdown('### r/formula1point5 submissions:'), f15_submissions_df.head(n))
    display(Markdown('### r/formula1point5 comments:'), f15_comments_df.head(n))

### r/formula1 submissions:

,created_utc,selftext,score,author,id,title,gilded
0,2022-06-01 12:00:41,[removed],1,[deleted],v2fbpg,[Discussion] Could professional ESports drivers drive a real F1 car? How realistic are the sims?,0
1,2022-06-01 12:07:50,[removed],2,Doomaster14,v2fh6w,Questions concerning Alonso's future,0
2,2022-06-01 12:15:14,,1393,motorace_addict,v2fmeh,Verstappen now has as many poles as Leclerc - but six times as many wins | 2022 Monaco Grand Prix stats and facts,0
3,2022-06-01 12:23:16,,161,MrTuxedo1,v2frea,Perez wins as Red Bull delivers race strategy blow to Ferrari - Mika Häkkinen’s thoughts on the Monaco Grand Prix,0


### r/formula1 comments:

,created_utc,score,author,id,gilded,body
0,2022-06-01 00:00:57,1,CowsWantToKillMe,iaq4tev,0,top part of the wing got shaken off in the tunnel.
1,2022-06-01 00:01:15,0,doc_55lk,iaq4urr,0,That's been the rumour with Mercedes lately cuz in previous seasons Bottas hasn't been the luckiest.
2,2022-06-01 00:01:41,3,Organic-Measurement2,iaq4wpz,0,"Ah well, it's looking great already!"
3,2022-06-01 00:01:46,10,not_right,iaq4x1h,0,And Ferrari would get them all wrong.


### r/formula1point5 submissions:

,created_utc,selftext,score,author,id,title,gilded
0,2022-06-07 09:21:41,,1,orfeomclaren,v6qyud,Formula 1 - Hakkinen vs Schumacher - Spa-Francorchamps 2000,0
1,2022-06-07 13:26:25,,1,orfeomclaren,v6viae,Formula 1 2003 - Rd 2 - Malaysian Grand Prix [Highlights] - Kimi Raikkonen Maiden Win,0
2,2022-06-09 08:12:22,,1,orfeomclaren,v8bwj6,Formula 1 2003 - Rd 9 - European Grand Prix (Nurburgring) [Highlights],0
3,2022-06-09 11:48:11,,1,ms_creativity,v8f1dk,Red Bull drivers free to fight each other,0


### r/formula1point5 comments:

,created_utc,score,author,id,gilded,body
0,2022-06-01 03:50:49,3,debrek,iaqwofj,0,What is your team name please?
1,2022-06-01 05:54:28,2,IgnisVizsla,iar7xgu,0,"It's lazily named team F1.5 and my name there is the same as my username here (Ignis Vizsla), I'm 34th on the leaderboard there for reference"
2,2022-06-01 06:20:29,3,debrek,iar9z0m,0,I had removed you as I thought you were inactive since you had a number of teams with an invalid team. I re-added you to the list.
3,2022-06-01 06:49:13,3,IgnisVizsla,iarc3x7,0,"Yeah that's my fault, I forgot to update my team after the rules changed as I always remembered only after quali and that was too late, I finally changed before Monaco though"


In [52]:
n = 3

with display_full_dataframe():
    display(Markdown('### r/formula1 posts:'), f1_df.head(n))
    display(Markdown('### r/formula1point5 posts:'), f15_df.head(n))

### r/formula1 posts:

,created_utc,score,author,id,gilded,text
0,2022-06-01 12:00:41,1,[deleted],v2fbpg,0,[Discussion] Could professional ESports drivers drive a real F1 car? How realistic are the sims? [removed]
1,2022-06-01 12:07:50,2,Doomaster14,v2fh6w,0,Questions concerning Alonso's future. [removed]
2,2022-06-01 12:15:14,1393,motorace_addict,v2fmeh,0,Verstappen now has as many poles as Leclerc - but six times as many wins | 2022 Monaco Grand Prix stats and facts.


### r/formula1point5 posts:

,created_utc,score,author,id,gilded,text
0,2022-06-07 09:21:41,1,orfeomclaren,v6qyud,0,Formula 1 - Hakkinen vs Schumacher - Spa-Francorchamps 2000.
1,2022-06-07 13:26:25,1,orfeomclaren,v6viae,0,Formula 1 2003 - Rd 2 - Malaysian Grand Prix [Highlights] - Kimi Raikkonen Maiden Win.
2,2022-06-09 08:12:22,1,orfeomclaren,v8bwj6,0,Formula 1 2003 - Rd 9 - European Grand Prix (Nurburgring) [Highlights]


# Baseline: Rule-Based Prediction Extraction

In [53]:
# TODO:

# Fastf1 historical data

In [ ]:
full_schedule = fastf1.get_event_schedule(dataset_constants.YEAR)
schedule = typing.cast(
    fastf1_events.EventSchedule,
    full_schedule[
        (full_schedule['EventDate'] >= dataset_constants.START_DATE) &
        (full_schedule['EventDate'] <= dataset_constants.END_DATE) &
        (full_schedule['EventFormat'] == 'conventional') # TODO: Skip sprint weekends for now. Also include sprint weekends later
    ],
)

with display_full_dataframe():
    display(schedule.iloc[-3:])

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
9,8,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2022,2022-06-12,Azerbaijan Grand Prix,conventional,Practice 1,2022-06-10 15:00:00+04:00,2022-06-10 11:00:00,Practice 2,2022-06-10 18:00:00+04:00,2022-06-10 14:00:00,Practice 3,2022-06-11 15:00:00+04:00,2022-06-11 11:00:00,Qualifying,2022-06-11 18:00:00+04:00,2022-06-11 14:00:00,Race,2022-06-12 15:00:00+04:00,2022-06-12 11:00:00,True
10,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2022,2022-06-19,Canadian Grand Prix,conventional,Practice 1,2022-06-17 14:00:00-04:00,2022-06-17 18:00:00,Practice 2,2022-06-17 17:00:00-04:00,2022-06-17 21:00:00,Practice 3,2022-06-18 13:00:00-04:00,2022-06-18 17:00:00,Qualifying,2022-06-18 16:00:00-04:00,2022-06-18 20:00:00,Race,2022-06-19 14:00:00-04:00,2022-06-19 18:00:00,True
11,10,Great Britain,Silverstone,FORMULA 1 LENOVO BRITISH GRAND PRIX 2022,2022-07-03,British Grand Prix,conventional,Practice 1,2022-07-01 13:00:00+01:00,2022-07-01 12:00:00,Practice 2,2022-07-01 16:00:00+01:00,2022-07-01 15:00:00,Practice 3,2022-07-02 12:00:00+01:00,2022-07-02 11:00:00,Qualifying,2022-07-02 15:00:00+01:00,2022-07-02 14:00:00,Race,2022-07-03 15:00:00+01:00,2022-07-03 14:00:00,True
13,12,France,Le Castellet,FORMULA 1 LENOVO GRAND PRIX DE FRANCE 2022,2022-07-24,French Grand Prix,conventional,Practice 1,2022-07-22 14:00:00+02:00,2022-07-22 12:00:00,Practice 2,2022-07-22 17:00:00+02:00,2022-07-22 15:00:00,Practice 3,2022-07-23 13:00:00+02:00,2022-07-23 11:00:00,Qualifying,2022-07-23 16:00:00+02:00,2022-07-23 14:00:00,Race,2022-07-24 15:00:00+02:00,2022-07-24 13:00:00,True
14,13,Hungary,Budapest,FORMULA 1 ARAMCO MAGYAR NAGYDÍJ 2022,2022-07-31,Hungarian Grand Prix,conventional,Practice 1,2022-07-29 14:00:00+02:00,2022-07-29 12:00:00,Practice 2,2022-07-29 17:00:00+02:00,2022-07-29 15:00:00,Practice 3,2022-07-30 13:00:00+02:00,2022-07-30 11:00:00,Qualifying,2022-07-30 16:00:00+02:00,2022-07-30 14:00:00,Race,2022-07-31 15:00:00+02:00,2022-07-31 13:00:00,True
15,14,Belgium,Spa-Francorchamps,FORMULA 1 ROLEX BELGIAN GRAND PRIX 2022,2022-08-28,Belgian Grand Prix,conventional,Practice 1,2022-08-26 14:00:00+02:00,2022-08-26 12:00:00,Practice 2,2022-08-26 17:00:00+02:00,2022-08-26 15:00:00,Practice 3,2022-08-27 13:00:00+02:00,2022-08-27 11:00:00,Qualifying,2022-08-27 16:00:00+02:00,2022-08-27 14:00:00,Race,2022-08-28 15:00:00+02:00,2022-08-28 13:00:00,True
16,15,Netherlands,Zandvoort,FORMULA 1 HEINEKEN DUTCH GRAND PRIX 2022,2022-09-04,Dutch Grand Prix,conventional,Practice 1,2022-09-02 12:30:00+02:00,2022-09-02 10:30:00,Practice 2,2022-09-02 16:00:00+02:00,2022-09-02 14:00:00,Practice 3,2022-09-03 12:00:00+02:00,2022-09-03 10:00:00,Qualifying,2022-09-03 15:00:00+02:00,2022-09-03 13:00:00,Race,2022-09-04 15:00:00+02:00,2022-09-04 13:00:00,True
17,16,Italy,Monza,FORMULA 1 PIRELLI GRAN PREMIO D’ITALIA 2022,2022-09-11,Italian Grand Prix,conventional,Practice 1,2022-09-09 14:00:00+02:00,2022-09-09 12:00:00,Practice 2,2022-09-09 17:00:00+02:00,2022-09-09 15:00:00,Practice 3,2022-09-10 13:00:00+02:00,2022-09-10 11:00:00,Qualifying,2022-09-10 16:00:00+02:00,2022-09-10 14:00:00,Race,2022-09-11 15:00:00+02:00,2022-09-11 13:00:00,True
18,17,Singapore,Marina Bay,FORMULA 1 SINGAPORE AIRLINES SINGAPORE GRAND PRIX 2022,2022-10-02,Singapore Grand Prix,conventional,Practice 1,2022-09-30 18:00:00+08:00,2022-09-30 10:00:00,Practice 2,2022-09-30 21:00:00+08:00,2022-09-30 13:00:00,Practice 3,2022-10-01 18:00:00+08:00,2022-10-01 10:00:00,Qualifying,2022-10-01 21:00:00+08:00,2022-10-01 13:00:00,Race,2022-10-02 20:00:00+08:00,2022-10-02 12:00:00,True
19,18,Japan,Suzuka,FORMULA 1 HONDA JAPANESE GRAND PRIX 2022,2022-10-09,Japanese Grand Prix,conventional,Practice 1,2022-10-07 12:00:00+09:00,2022-10-07 03:00:00,Pra

In [55]:
posts_df = f1_df
race_weekend = schedule.iloc[-1]
first_post_at = typing.cast(dt.datetime, race_weekend['Session1DateUtc']) - dt.timedelta(days=1)
last_post_at = typing.cast(dt.datetime, race_weekend['Session5DateUtc'])
posts_df = posts_df[
    (posts_df['created_utc'] >= first_post_at) &
    (posts_df['created_utc'] <= last_post_at)
]

def get_top20(race_weekend: fastf1_events.Event) -> pd.DataFrame:
    race_session = race_weekend.get_session('Race')
    race_session.load(laps=False, telemetry=False, weather=False, messages=False)
    top20 = race_session.results[['FullName', 'Position']].astype({'Position': np.uint8})
    return top20

top20s = tuple(
    get_top20(typing.cast(fastf1_events.Event, race_weekend))
    for _, race_weekend in schedule.iterrows()
)
display(hide_index(top20s[-1]))

FullName,Position
Max Verstappen,1
Charles Leclerc,2
Sergio Perez,3
Carlos Sainz,4
George Russell,5
Lando Norris,6
Esteban Ocon,7
Lance Stroll,8
Daniel Ricciardo,9
Sebastian Vettel,10


# Pre-trained models

# Sentiment score:

In [56]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
from scipy.special import softmax

def driver_sentiment(comments, driver_list, scores):
    model_name = "yangheng/deberta-v3-base-absa-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    results = {driver: {"positive": 0.0, "neutral": 0.0, "negative": 0.0, "count": 0} for driver in driver_list}

    for comment, score in zip(comments, scores):
        found_drivers = [driver for driver in driver_list if driver in comment]
        
        for aspect in found_drivers:
            inputs = tokenizer(comment, aspect, return_tensors="pt", truncation=True, padding=True).to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
            
            sent = outputs.logits[0].cpu().numpy()
            probabilities = softmax(sent)

            results[aspect]["positive"] += probabilities[2] * score
            results[aspect]["neutral"] += probabilities[1] * score
            results[aspect]["negative"] += probabilities[0] * score
            results[aspect]["count"] += score

    for driver, sentiment in results.items():
        if sentiment["count"] > 0:
            sentiment["positive"] /= sentiment["count"]
            sentiment["neutral"] /= sentiment["count"]
            sentiment["negative"] /= sentiment["count"]

    return results


prediction_posts_df = ['Carlos Sainz is loving this upgraded car, good top 3 for the race tomorrow! I disagree with you, Max Verstappen will definitely finish first. I think BOT will finish behind NOR, who will probably finish 7th. That\'s my opinion at least... I predict that the RedBulls with finish 1-2. Nah, the Danish driver from Haas will almost certainly finish in points! Stroll on the podium and Vettel in points. I like cookies!']
driver_list = ['Carlos Sainz', 'Max Verstappen']

F1_names= {
    'max verstappen',
    'charles leclerc',
    'sergio perez',
    'george russell',
    'carlos sainz',
    'lewis hamilton',
    'lando norris',
    'esteban ocon',
    'fernando alonso',
    'valtteri bottas',
    'daniel ricciardo',
    'sebastian vettel',
    'kevin magnussen',
    'pierre gasly',
    'lance stroll',
    'mick schumacher',
    'yuki tsunoda',
    'zhou guanyu',
    'alexander albon',
    'nicholas latifi',
    'nyck de vries',
    'nico hulkenberg',
    'oscar piastri',
    'liam lawson',
    'logan sargeant'
}

posts_df = load_f1_df(1000)
posts_df['text'] = posts_df['text'].apply(preprocessing.correct_spelling_in_text_spacy)
comments = posts_df["text"].tolist()
scores = posts_df["score"].tolist()


results = driver_sentiment(comments, F1_names, scores)
print(results)
for driver, sentiment in results.items():
    if sentiment["count"] > 0:
        print(f"{driver}: [Positive: {sentiment['positive']:.4f}, Neutral: {sentiment['neutral']:.4f}, Negative: {sentiment['negative']:.4f}]")
    else:
        print(f"{driver}: No mentions found.")

c:\Users\raf\Documents\GitHub\f1-subreddits-nlp\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'lewis hamilton': {'positive': 0.13853888933219438, 'neutral': 0.4295732431285516, 'negative': 0.43188787317419325, 'count': 15030}, 'fernando alonso': {'positive': 0.547514488389884, 'neutral': 0.3375483624678872, 'negative': 0.11493712610482584, 'count': 10307}, 'george russell': {'positive': 0.039409447393855934, 'neutral': 0.6390031803903622, 'negative': 0.3215873683707211, 'count': 4580}, 'max verstappen': {'positive': 0.10685420350202213, 'neutral': 0.8168497674411428, 'negative': 0.07629602186731979, 'count': 133400}, 'nyck de vries': {'positive': 0.7035142779350281, 'neutral': 0.2487473487854004, 'negative': 0.047738347202539444, 'count': 251}, 'daniel ricciardo': {'positive': 0.2476961301436094, 'neutral': 0.5786894359994762, 'negative': 0.17361445376296974, 'count': 9051}, 'valtteri bottas': {'positive': 0.02525833082442048, 'neutral': 0.9565289607379388, 'negative': 0.018212692842905793, 'count': 6391}, 'lance stroll': {'positive': 0.015630004871924227, 'neutral': 0.5473516

In [57]:
def final_scores(results):
    final_scores = []

    for driver, sentiment in results.items():
        if sentiment["count"] > 0:
            sentiment_score = (sentiment["positive"] - sentiment["negative"])
            final_scores.append((driver.title(), sentiment_score))

    # Sort drivers by positive - negative score (descending order)
    final_scores.sort(key=lambda x: x[1], reverse=True)
    
    return final_scores

# Print sorted results
scores = final_scores(results)

print("Drivers ranked by (positive - negative score):")
for driver, score in scores:
    print(f"{driver}: {score:.4f}")


Drivers ranked by (positive - negative score):
Nyck De Vries: 0.6558
Alexander Albon: 0.6544
Fernando Alonso: 0.4326
Sergio Perez: 0.1787
Oscar Piastri: 0.1527
Daniel Ricciardo: 0.0741
Lando Norris: 0.0729
Max Verstappen: 0.0306
Yuki Tsunoda: 0.0159
Valtteri Bottas: 0.0070
Nicholas Latifi: 0.0069
Zhou Guanyu: -0.0067
Pierre Gasly: -0.0267
Esteban Ocon: -0.0382
Kevin Magnussen: -0.0401
Nico Hulkenberg: -0.0428
Mick Schumacher: -0.0473
Charles Leclerc: -0.0943
Sebastian Vettel: -0.2150
Carlos Sainz: -0.2446
George Russell: -0.2822
Lewis Hamilton: -0.2933
Lance Stroll: -0.4214
Liam Lawson: -0.4617
Logan Sargeant: -0.5681


In [72]:
def prediction(n_event, final_scores, n_events=5, historical_score_contribution=0.4):
    #func to import historical data for this race
    historical_data_event = get_top20(full_schedule.iloc[n_event])
    historical_scores = {row["FullName"]: 0 for _, row in historical_data_event.iterrows()}

    for i in range(n_events):
        #func to import historical data for one of the last 5 races
        historical_data = get_top20(full_schedule.iloc[n_event - (i+1)])

        for _, row in historical_data.iterrows():
            if row["FullName"] not in historical_scores:
                historical_scores[row["FullName"]] = 0  # Initialize missing driver
            historical_scores[row["FullName"]] += 1 - ((row["Position"] - 1) / 19) * 2

    for driver, score in historical_scores.items():
        historical_scores[driver] = score / n_events

    final_scores_dict = dict(final_scores)

    final_prediction = []
    for driver, historical_score in historical_scores.items():
        if driver in final_scores_dict:
            score = final_scores_dict[driver]
            
            combined_score = (1 - historical_score_contribution) * score + historical_score_contribution * historical_score
            final_prediction.append((driver, combined_score, score, historical_score))
        else:
            final_prediction.append((driver, historical_score, None, historical_score))

    final_prediction.sort(key=lambda x: x[1], reverse=True)
    final_prediction_dict = {
        driver: {"combined_score": comb_score, "sentiment_score": pred_score, "historical_score": hist_score}
        for driver, comb_score, pred_score, hist_score in final_prediction
    }
    # Calculate MAE
    # Map drivers to their predicted positions
    predicted_positions = {driver: i + 1 for i, (driver, _, _, _) in enumerate(final_prediction)}

    # Map drivers to their true positions
    true_positions = {row["FullName"]: row["Position"] for _, row in historical_data_event.iterrows()}

    # Compute absolute errors for drivers present in both sets
    data = []
    for driver, predicted_position in predicted_positions.items():
        true_position = true_positions.get(driver, None)
        scores = final_prediction_dict.get(driver, {"combined_score": None, "sentiment_score": None, "historical_score": None})

        error = abs(predicted_position - true_position) if true_position is not None else None
        data.append({
            "driver_name": driver,
            "predicted_position": predicted_position,
            "true_position": true_position,
            "error": error,
            "combined_score": scores["combined_score"],
            "sentiment_score": scores["sentiment_score"],
            "historical_score": scores["historical_score"]
        })

    prediction_df = pd.DataFrame(data)

    return prediction_df

index = 9
prediction_df = prediction(index, scores, n_events=5, historical_score_contribution=0.4)
display(prediction_df)
mae = prediction_df["error"].mean()
print(mae)
position = 0
# for driver, score in final_prediction:
#     position += 1
#     print(f"{driver} finishes in position:{position}      {score:.4f}")

,driver_name,predicted_position,true_position,error,combined_score,sentiment_score,historical_score
0,Charles Leclerc,1,19,18,0.359033,0.324704,0.410526
1,Sebastian Vettel,2,6,4,0.288644,0.656511,-0.263158
2,Carlos Sainz,3,20,17,0.210026,0.300921,0.073684
3,Max Verstappen,4,1,3,0.139215,-0.167976,0.600000
4,Sergio Perez,5,2,3,0.115176,-0.390496,0.873684
5,George Russell,6,3,3,0.083138,-0.317576,0.684211
6,Valtteri Bottas,7,11,4,0.032897,-0.190786,0.368421
7,Lewis Hamilton,8,4,4,-0.002002,-0.234915,0.347368
8,Esteban Ocon,9,10,1,-0.058379,-0.160456,0.094737
9,Lando Norris,10,9,1,-0.059756,-0.260997,0.242105


5.0


# GLiNER

In [59]:
import os
os.environ["HF_HOME"] = "C:\\cache"
from gliner import GLiNER

gliner_pickle_path = config.DATA_DIR / '.cache' / 'gliner_model.pkl'
gliner_pickle_path.parent.mkdir(parents=True, exist_ok=True)
use_cache = False

if use_cache:
    if not gliner_pickle_path.exists():
        gliner_model = GLiNER.from_pretrained('urchade/gliner_medium-v2.1')

        with open(gliner_pickle_path, 'wb') as file:
            pickle.dump(gliner_model, file)
    else:
        with open(gliner_pickle_path, 'rb') as file:
            gliner_model = pickle.load(file)
else:
    gliner_model = GLiNER.from_pretrained('urchade/gliner_medium-v2.1')

gliner_model.to(DEVICE);

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\raf\Documents\GitHub\f1-subreddits-nlp\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [60]:
print(type(gliner_model))

<class 'gliner.model.GLiNER'>


In [61]:
nlp = spacy.load('en_core_web_sm')
text = 'Carlos Sainz is loving this upgraded car, good top 3 for the race tomorrow! I disagree with you, verstappening will definitely finish first. I think BOT will finish behind NOR, who will probably finish 7th. That\'s my opinion at least... I predict that the RedBulls with finish 1-2. Nah, the Danish driver from Haas will almost certainly finish in points! Stroll on the podium and Vettel in points. I like cookies!'
debug = True

if debug:
    doc = nlp(text)
    df = pd.DataFrame({'text': tuple(sentence.text for sentence in doc.sents)})
else:
    df = load_f1_df(1000)

In [62]:
with display_full_dataframe():
    display(df.head())

def has_prediction(post_text: str, threshold: float = 0.45) -> bool:
    # doc = nlp(post_text)

    # TODO: does GLiNER's performance improve with more context? if yes, refactor to chunking instead of going over each sentence individually
    # for sentence in doc.sents:
    # TODO: for some reason, if you include only 'position', the predictions are far worse than with 'driver' included
    with torch.no_grad():
        entities = gliner_model.predict_entities(post_text, ('driver', 'position',), threshold=threshold) # TODO: very low threshold
    position_entities = tuple(entity for entity in entities if entity['label'] == 'position')

    if debug:
        print(position_entities)
        print(tuple(position['text'] for position in position_entities))

    if len(position_entities) != 0:
        return True

    return False


predictions_df = df[df['text'].apply(has_prediction)]
print(len(predictions_df))
with display_full_dataframe():
    display(predictions_df.head())

,text
0,"Carlos Sainz is loving this upgraded car, good top 3 for the race tomorrow!"
1,"I disagree with you, verstappening will definitely finish first."
2,"I think BOT will finish behind NOR, who will probably finish 7th."
3,That's my opinion at least...
4,I predict that the RedBulls with finish 1-2.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


({'start': 47, 'end': 52, 'text': 'top 3', 'label': 'position', 'score': 0.6188601851463318},)
('top 3',)
()
()
({'start': 61, 'end': 64, 'text': '7th', 'label': 'position', 'score': 0.8359587788581848},)
('7th',)
()
()
({'start': 40, 'end': 43, 'text': '1-2', 'label': 'position', 'score': 0.8649768829345703},)
('1-2',)
({'start': 65, 'end': 71, 'text': 'points', 'label': 'position', 'score': 0.5379745960235596},)
('points',)
({'start': 14, 'end': 20, 'text': 'podium', 'label': 'position', 'score': 0.6982569694519043}, {'start': 35, 'end': 41, 'text': 'points', 'label': 'position', 'score': 0.4657585620880127})
('podium', 'points')
()
()
5


,text
0,"Carlos Sainz is loving this upgraded car, good top 3 for the race tomorrow!"
2,"I think BOT will finish behind NOR, who will probably finish 7th."
4,I predict that the RedBulls with finish 1-2.
5,"Nah, the Danish driver from Haas will almost certainly finish in points!"
6,Stroll on the podium and Vettel in points.


In [63]:
# def has_prediction_dask(post_text):
#     return has_prediction(post_text)

In [64]:
# import dask.dataframe as dd

# # Convert the Pandas DataFrame to a Dask DataFrame
# dask_df = dd.from_pandas(load_f1_df(10), npartitions=16)  # Adjust the number of partitions as needed

# # Apply the function in parallel
# dask_df['has_prediction'] = dask_df['text'].map(has_prediction_dask, meta=('text', 'bool'))

# # Compute the result and convert back to a Pandas DataFrame
# result_df = dask_df[dask_df['has_prediction']].compute()

# # Display the filtered DataFrame
# with display_full_dataframe():
#     display(result_df.head())
historical_data_event = get_top20(full_schedule.iloc[index])
true_positions = {row["FullName"]: row["Position"] for _, row in historical_data_event.iterrows()}
display(true_positions)

{'Max Verstappen': 1,
 'Sergio Perez': 2,
 'George Russell': 3,
 'Lewis Hamilton': 4,
 'Pierre Gasly': 5,
 'Sebastian Vettel': 6,
 'Fernando Alonso': 7,
 'Daniel Ricciardo': 8,
 'Lando Norris': 9,
 'Esteban Ocon': 10,
 'Valtteri Bottas': 11,
 'Alexander Albon': 12,
 'Yuki Tsunoda': 13,
 'Mick Schumacher': 14,
 'Nicholas Latifi': 15,
 'Lance Stroll': 16,
 'Kevin Magnussen': 17,
 'Guanyu Zhou': 18,
 'Charles Leclerc': 19,
 'Carlos Sainz': 20}

In [75]:
text = 'Carlos Sainz is loving this upgraded car, good top 3 for the race tomorrow! I disagree with you, verstappening will definitely finish first. I think BOT will finish behind NOR, who will probably finish 7th. That\'s my opinion at least... I predict that the RedBulls with finish 1-2. Nah, the Danish driver from Haas will almost certainly finish in points! Stroll on the podium and Vettel in points. I like cookies!'
debug = False
import dask.dataframe as dd
import warnings
warnings.filterwarnings(
    "ignore",
    message="Sentence of length .* has been truncated to .*",
    category=UserWarning
)

if debug:
    doc = nlp(text)
    posts_df = pd.DataFrame({'text': tuple(sentence.text for sentence in doc.sents)})
else:
    start = time.perf_counter_ns()
    all_posts_df = load_f1_df()
    end = time.perf_counter_ns()
    print((end - start) / 10 ** 9, "load time")

# with display_full_dataframe():
#     display(hide_index(df.head()))

# df['text'] = df['text'].apply(preprocessing.correct_spelling_in_text_spacy)
# df = df[df['text'].apply(has_prediction)]

def display_posts_df(n=3):
    global posts_df
    
    with display_full_dataframe():
        display(hide_index(posts_df.head(n)))

predictions_dict = {}
mae_list = []

for index, race_weekend in schedule.iterrows():
    
    print(index, race_weekend)
    #load relevant post
    first_post_at = typing.cast(dt.datetime, race_weekend['Session1DateUtc']) #- dt.timedelta(days=1)
    last_post_at = typing.cast(dt.datetime, race_weekend['Session5DateUtc'])
    posts_df = all_posts_df[
        (all_posts_df['created_utc'] >= first_post_at) &
        (all_posts_df['created_utc'] <= last_post_at)
    ]
    print('number of posts for event: ', len(posts_df))

    # spelling correction
    start = time.perf_counter_ns()
    posts_df['text'] = posts_df['text'].apply(preprocessing.correct_spelling_in_text_spacy)
    end = time.perf_counter_ns()
    print((end - start) / 10 ** 9, "spell time")

    # only predictions
    start = time.perf_counter_ns()
    
    posts_df = posts_df[posts_df['text'].apply(has_prediction)]
    print('number of posts with prediction: ', len(posts_df))

    end = time.perf_counter_ns()
    print((end - start) / 10 ** 9, "filter time")

    # sentiment score
    start = time.perf_counter_ns()
    comments = posts_df["text"].tolist()
    upvotes = posts_df["score"].tolist()
    sentiment = driver_sentiment(comments, F1_names, upvotes)
    scores = final_scores(sentiment)
    end = time.perf_counter_ns()
    print((end - start) / 10 ** 9, "sentiment time")

    # final prediction
    start = time.perf_counter_ns()
    prediction_df = prediction(index, scores, n_events=5, historical_score_contribution=0.4)
    mae = prediction_df["error"].mean()
    display(prediction_df)
    print("the MAE for predicted vs true position is: ", mae)
    end = time.perf_counter_ns()
    print((end - start) / 10 ** 9, "final pred time")

    predictions_dict[index] = prediction_df
    print("\n" * 3)
    

with pd.ExcelWriter(config.DATA_DIR /"predictions.xlsx") as writer:
    for key, df in predictions_dict.items():
        df.to_excel(writer, sheet_name=f"Iteration_{key}", index=False)

# loaded_predictions = pd.read_excel(config.DATA_DIR /"predictions.xlsx", sheet_name=None)  # Returns a dictionary of DataFrames



91.6175755 load time
9 RoundNumber                                             8
Country                                        Azerbaijan
Location                                             Baku
OfficialEventName    FORMULA 1 AZERBAIJAN GRAND PRIX 2022
EventDate                             2022-06-12 00:00:00
EventName                           Azerbaijan Grand Prix
EventFormat                                  conventional
Session1                                       Practice 1
Session1Date                    2022-06-10 15:00:00+04:00
Session1DateUtc                       2022-06-10 11:00:00
Session2                                       Practice 2
Session2Date                    2022-06-10 18:00:00+04:00
Session2DateUtc                       2022-06-10 14:00:00
Session3                                       Practice 3
Session3Date                    2022-06-11 15:00:00+04:00
Session3DateUtc                       2022-06-11 11:00:00
Session4                                       Qu

R:\Temp\Raf\ipykernel_10044\2565367918.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_df['text'] = posts_df['text'].apply(preprocessing.correct_spelling_in_text_spacy)


484.4104127 spell time
number of posts with prediction:  4940
1487.9600493 filter time


c:\Users\raf\Documents\GitHub\f1-subreddits-nlp\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


93.9827281 sentiment time


...

357.9886337 spell time
number of posts with prediction:  4098
1086.479302 filter time


c:\Users\raf\Documents\GitHub\f1-subreddits-nlp\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


85.0607304 sentiment time


,driver_name,predicted_position,true_position,error,combined_score,sentiment_score,historical_score
0,Max Verstappen,1,1,0,0.318833,0.019108,0.768421
1,Sergio Perez,2,3,1,0.259843,-0.065173,0.747368
2,Lance Stroll,3,8,5,0.160793,0.387287,-0.178947
3,Charles Leclerc,4,2,2,0.158724,-0.219671,0.726316
4,George Russell,5,5,0,0.045456,-0.211959,0.431579
5,Esteban Ocon,6,7,1,0.027094,-0.032037,0.115789
6,Daniel Ricciardo,7,9,2,0.026455,0.135320,-0.136842
7,Valtteri Bottas,8,15,7,0.010971,0.179688,-0.242105
8,Fernando Alonso,9,20,11,-0.001076,0.047329,-0.073684
9,Lando Norris,10,6,4,-0.003051,-0.082278,0.115789


the MAE for predicted vs true position is:  3.8
11.7538665 final pred time






In [70]:
# nececary: pip install openpyxl
with pd.ExcelWriter(config.DATA_DIR /"predictions.xlsx") as writer:
    for key, df in predictions_dict.items():
        df.to_excel(writer, sheet_name=f"Iteration_{key}", index=False)

In [ ]:
print(predictions_dict)

In [ ]:
# sentiment score
start = time.perf_counter_ns()
comments = posts_df["text"].tolist()
upvotes = posts_df["score"].tolist()
sentiment = driver_sentiment(comments, F1_names, upvotes)
scores = final_scores(sentiment)
end = time.perf_counter_ns()
print((end - start) / 10 ** 9, "sentiment time")
# print("Drivers ranked by (positive - negative score):")
# for driver, score in scores:
#     print(f"{driver}: {score:.4f}")

# final prediction
start = time.perf_counter_ns()
pred = prediction(index, scores, n_events=5, historical_score_contribution=0.75)
end = time.perf_counter_ns()
print((end - start) / 10 ** 9, "final pred time")


predictions_dict[index] = pred
position = 0
for driver, score in pred:
    position += 1
    print(f"{driver} finishes in position:{position}      {score:.4f}")

In [78]:
validation_posts = load_f1_df(2000)['text']
display(validation_posts.head())
validation_posts = validation_posts.apply(preprocessing.correct_spelling_in_text_spacy)
validation_posts = validation_posts[validation_posts.apply(has_prediction)]
print(len(validation_posts))
with display_full_dataframe():
    display(validation_posts.head())

validation_posts.to_csv(config.DATA_DIR /'file1.csv')
    

0    [Discussion] Could professional ESports driver...
1      Questions concerning Alonso's future. [removed]
2    Verstappen now has as many poles as Leclerc - ...
3    Perez wins as Red Bull delivers race strategy ...
4    The "new" qualifying since (I think 2021?) 202...
Name: text, dtype: object

435


5                                  [joe award] sources saying that peter layer has gone from his position as head of F1 at ﻿the max verstappen...checking now. 
13                                                                                                       2022 monaco of odis F1 face debrief - mercedes-AMGF1. 
34    remember this team, check and esteban ocon used to be star for this team before ﻿the downfall. a bought this limited edition in 2012 INDIAN of. [deleted]
38                                                                   wolff explains why mercedes ton't a-turn on W13 stuck in 'no man's land' | RacingNews365. 
40                         sebastian vettel delighted not to leave monaco 'empty handed' after P10 finish, as lance stroll takes positives from 'tricky race ' 
Name: text, dtype: object

In [87]:
# add new collumn with the label for each driver in the form: {'nyck de vries': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'alexander albon': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'yuki tsunoda': {'positive': 0.0057728992807760575, 'neutral': 0.9906835493288542, 'negative': 0.0035435524033872704, 'count': 57}, 'nicholas latifi': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'lance stroll': {'positive': 0.0025615381891839206, 'neutral': 0.8542155921459198, 'negative': 0.1432228833436966, 'count': 2}, 'sergio perez': {'positive': 0.19108588388694617, 'neutral': 0.7905566372219701, 'negative': 0.018357482762380587, 'count': 19643}, 'liam lawson': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'logan sargeant': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'oscar piastri': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'kevin magnussen': {'positive': 0.3014230728149414, 'neutral': 0.13589109480381012, 'negative': 0.5626858472824097, 'count': 119}, 'charles leclerc': {'positive': 0.3298702842287487, 'neutral': 0.6215965418101896, 'negative': 0.04853320448722903, 'count': 5804}, 'lewis hamilton': {'positive': 0.19249720936472856, 'neutral': 0.6415854867461116, 'negative': 0.1659173017253239, 'count': 602}, 'max verstappen': {'positive': 0.027957634713532145, 'neutral': 0.86048945951617, 'negative': 0.11155290395479907, 'count': 37483}, 'lando norris': {'positive': 0.10887042551651653, 'neutral': 0.8807938621942907, 'negative': 0.010335699243584025, 'count': 7665}, 'sebastian vettel': {'positive': 0.21298285713947993, 'neutral': 0.3937424868611979, 'negative': 0.39327468428606643, 'count': 9777}, 'mick schumacher': {'positive': 0.22496380869912483, 'neutral': 0.172746341228788, 'negative': 0.6022898651057537, 'count': 123}, 'fernando alonso': {'positive': 0.6737411749587325, 'neutral': 0.30806004730469894, 'negative': 0.018198754213155918, 'count': 8161}, 'zhou guanyu': {'positive': 0.003083703340962529, 'neutral': 0.9933833479881287, 'negative': 0.0035329661332070827, 'count': 5230}, 'nico hulkenberg': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'carlos sainz': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'esteban ocon': {'positive': 0.0, 'neutral': 0.0, 'negative': 0.0, 'count': 0}, 'daniel ricciardo': {'positive': 0.2623085604583092, 'neutral': 0.7047550600192255, 'negative': 0.03293641074422787, 'count': 5407}, 'valtteri bottas': {'positive': 0.0030204161646588666, 'neutral': 0.9952852886277141, 'negative': 0.0016942658217249356, 'count': 383}, 'george russell': {'positive': 0.019467041386267196, 'neutral': 0.24923343850033625, 'negative': 0.7312994546244864, 'count': 1848}, 'pierre gasly': {'positive': 0.15713243364682997, 'neutral': 0.7395840842833875, 'negative': 0.10328345954387029, 'count': 17877}}
# load
chatgpt_df = pd.read_csv(config.DATA_DIR /'file1.csv')
print(chatgpt_df)

for gpt_df in chatgpt_df["text"]:
    print(i, gpt_df)
#     comment = gpt_df["text"].iloc[i]
#     score = gpt_df["score"].iloc[i]

#     results = driver_sentiment(comment, F1_names, score)
#     # compare chatgpt column with results
#     # mae = abs()


     Unnamed: 0                                               text
0             5  [joe award] sources saying that peter layer ha...
1            13  2022 monaco of odis F1 face debrief - mercedes...
2            34  remember this team, check and esteban ocon use...
3            38  wolff explains why mercedes ton't a-turn on W1...
4            40  sebastian vettel delighted not to leave monaco...
..          ...                                                ...
430        3945  &it; is speed is fairly decent in ﻿the rookie ...
431        3957  maybe a'm wrong, but isn't it more costly to R...
432        3964  looking at it from ﻿the front, somehow reminds...
433        3986  to you mean by thai? and yes, sergio perez did...
434        3989  &it; why make racing even more boring\n\nit do...

[435 rows x 2 columns]
1 [joe award] sources saying that peter layer has gone from his position as head of F1 at ﻿the max verstappen...checking now. 
1 2022 monaco of odis F1 face debrief - merce